# Day 2: 1202 Program Alarm

https://adventofcode.com/2019/day/2

An Intcode program is a list of integers separated by commas (like 1,0,0,3,99). 

To run one, start by looking at the first integer (called position 0). Here, you will find an opcode - **either 1, 2, or 99**. 

The opcode indicates what to do.

**Encountering an unknown opcode means something went wrong.**

## Opcodes

### Opcode **99** (HALT)

It means that the program is finished and should immediately halt. 

### Opcode **1** (ADD)

Adds together numbers read from two positions and stores the result in a third position. 

The three integers immediately after the opcode tell you these three positions:

    - the first two indicate the positions from which you should read the input values
    - the third indicates the position at which the output should be stored.

### Opcode **2** (MUL) 

It works exactly like **opcode 1**, except it multiplies the two inputs instead of adding them. 

Again, the three integers after the opcode indicate where the inputs and outputs are, not their values.

In [18]:
type Opcode =
    | ADD of int * int * int
    | MUL of int * int * int
    | HALT

In [25]:
type Memory = int array
type Address = int

let readInstruction (mem:Memory) (ptr:Address) : Opcode =
    match (mem.[ptr]) with
        |  1 -> ADD (mem.[ptr+1], mem.[ptr+2], mem.[ptr+3])
        |  2 -> MUL (mem.[ptr+1], mem.[ptr+2], mem.[ptr+3])
        | 99 -> HALT
        |  x -> failwith (sprintf "Invalid opcode %A" x)

## Opcode discrimination

_Asuming well formed memory_ 


In [31]:
readInstruction [| 1; 1; 2; 3 |] 0

Item1,Item2,Item3,Tag,IsADD,IsMUL,IsHALT
1,2,3,0,True,False,False


In [32]:
readInstruction [| 2; 1; 2; 3 |] 0

Item1,Item2,Item3,Tag,IsADD,IsMUL,IsHALT
1,2,3,1,False,True,False


In [29]:
readInstruction [| 99 |] 0

Tag,IsADD,IsMUL,IsHALT
2,False,False,True


**This should fail**

In [30]:
readInstruction [| 10 |] 0

Unhandled exception: Invalid opcode 10

## Implement Program execution
_Using mutable reference to memory for performance_

In [60]:
let rec runProgram (mem:Memory byref) (ptr:Address) : Unit =
    
    match (readInstruction mem ptr) with
        | ADD (i1,i2,o) -> 
            mem.[o] <- mem.[i1] + mem.[i2]    
            runProgram &mem (ptr+4)
        | MUL (i1,i2,o) -> 
            mem.[o] <- mem.[i1] * mem.[i2]
            runProgram &mem (ptr+4)
        | HALT -> ()

index,value
0,2
1,0
2,0
3,0
4,99


## Test cases

`1,0,0,0,99` becomes `2,0,0,0,99` (1 + 1 = 2).

In [62]:
let mutable memTest1 = [| 1; 0; 0; 0; 99 |]
runProgram &memTest1 0 
memTest1

index,value
0,2
1,0
2,0
3,0
4,99


`2,3,0,3,99` becomes `2,3,0,6,99` (3 * 2 = 6).

In [63]:
let mutable memTest2 = [| 2; 3; 0; 3; 99 |]
runProgram &memTest2 0 
memTest2

index,value
0,2
1,3
2,0
3,6
4,99


`2,4,4,5,99,0` becomes `2,4,4,5,99,9801` (99 * 99 = 9801).

In [65]:
let mutable memTest3 = [| 2; 4; 4; 5; 99; 0 |]
runProgram &memTest3 0 
memTest3

index,value
0,2
1,4
2,4
3,5
4,99
5,9801


`1,1,1,4,99,5,6,0,99` becomes `30,1,1,4,2,5,6,0,99`.

In [68]:
let mutable memTest4 = [| 1; 1; 1; 4; 99; 5; 6; 0; 99 |]
runProgram &memTest4 0 
memTest4

index,value
0,30
1,1
2,1
3,4
4,2
5,5
6,6
7,0
8,99


## Problem

Once you have a working computer, the first step is to restore the gravity assist program (your puzzle input) to the "1202 program alarm" state it had just before the last computer caught fire. 

To do this, before running the program, replace position 1 with the value 12 and replace position 2 with the value 2. 

What value is left at position 0 after the program halts?